In [5]:
# STEP 1: Import the necessary modules.
import os
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.tasks.python.vision import RunningMode

In [6]:
# STEP 2: Setup HandLandmarker configuration
hand_base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
face_base_options = python.BaseOptions(model_asset_path='face_landmarker.task')
hand_options = vision.HandLandmarkerOptions(base_options=hand_base_options,
                                            num_hands=2, running_mode=RunningMode.VIDEO)

face_options = vision.FaceLandmarkerOptions(base_options=face_base_options,
                                          num_faces=1, running_mode=RunningMode.VIDEO)

In [7]:
# STEP 4: Generate landmarks for each frame of every video
for root, _, files in os.walk("../food/data/train/zwiebel"):
    for filename in files:  # loop through files in the current directory
        capture = cv2.VideoCapture(os.path.join(root, filename))
        hand_detector = vision.HandLandmarker.create_from_options(hand_options)
        face_datector = vision.FaceLandmarker.create_from_options(face_options)
        frameNr = 0
        while True:
            ret, frame = capture.read()
            if not ret:
                break
            # Convert the frame to a MediaPipe Image.
            image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            # Detect hand landmarks in the current frame.
            hand_detection_result = hand_detector.detect_for_video(image, frameNr)
            face_detection_result = face_datector.detect_for_video(image, frameNr)
            hand_csv = []

            # Collect landmarks and handedness for each detected hand.
            for idx in range(len(hand_detection_result.hand_landmarks)):
                handedness = hand_detection_result.handedness[idx][0].category_name
                landmarks = hand_detection_result.hand_landmarks[idx]
                hand_csv.append("\n".join(map(lambda x: f"{handedness}, x:{x.x}, y:{x.y}, z:{x.z}", landmarks)))

            for idx in range(len(face_detection_result.face_landmarks)):
                landmarks = face_detection_result.face_landmarks[idx]
                hand_csv.append("\n".join(map(lambda x: f"face, x:{x.x}, y:{x.y}, z:{x.z}", landmarks)))

            # Write the landmarks to a CSV file.
            try:
                os.mkdir(os.path.join(root, "landmarks"))
            except FileExistsError:
                pass
            with open(os.path.join(root, f"landmarks/{filename}_landmarks.csv"), "a") as f:
                f.write(f"frame {frameNr} \n")
                f.write("\n".join(hand_csv))
                f.write("\n")

            frameNr = frameNr + 1
            print(f"Processed {filename} frame {frameNr}")
        capture.release()

Processed zwiebel_train_6.mp4 frame 1
Processed zwiebel_train_6.mp4 frame 2
Processed zwiebel_train_6.mp4 frame 3
Processed zwiebel_train_6.mp4 frame 4
Processed zwiebel_train_6.mp4 frame 5


OpenCV: Couldn't read video stream from file "../food/data/train/zwiebel/.DS_Store"
I0000 00:00:1750067660.007410 60479919 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1750067660.012317 60482324 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750067660.015991 60482324 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1750067660.017393 60479919 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1750067660.017659 60479919 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1750067660.020421 60482339 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0

Processed zwiebel_train_6.mp4 frame 6
Processed zwiebel_train_6.mp4 frame 7
Processed zwiebel_train_6.mp4 frame 8
Processed zwiebel_train_6.mp4 frame 9
Processed zwiebel_train_6.mp4 frame 10
Processed zwiebel_train_6.mp4 frame 11
Processed zwiebel_train_6.mp4 frame 12
Processed zwiebel_train_6.mp4 frame 13
Processed zwiebel_train_6.mp4 frame 14
Processed zwiebel_train_6.mp4 frame 15
Processed zwiebel_train_6.mp4 frame 16
Processed zwiebel_train_6.mp4 frame 17
Processed zwiebel_train_6.mp4 frame 18
Processed zwiebel_train_6.mp4 frame 19
Processed zwiebel_train_6.mp4 frame 20
Processed zwiebel_train_6.mp4 frame 21
Processed zwiebel_train_6.mp4 frame 22
Processed zwiebel_train_6.mp4 frame 23
Processed zwiebel_train_6.mp4 frame 24
Processed zwiebel_train_6.mp4 frame 25
Processed zwiebel_train_6.mp4 frame 26
Processed zwiebel_train_6.mp4 frame 27
Processed zwiebel_train_6.mp4 frame 28
Processed zwiebel_train_6.mp4 frame 29
Processed zwiebel_train_6.mp4 frame 30
Processed zwiebel_train_6.mp4

I0000 00:00:1750067661.946930 60479919 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1750067661.951580 60482490 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750067661.955674 60482496 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1750067661.960395 60479919 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1750067661.960636 60479919 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1750067661.963057 60482503 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750067661.969671 60482507 inference_feedback_manager.cc:114] Feedback man

Processed zwiebel_train_7.mp4 frame 6
Processed zwiebel_train_7.mp4 frame 7
Processed zwiebel_train_7.mp4 frame 8
Processed zwiebel_train_7.mp4 frame 9
Processed zwiebel_train_7.mp4 frame 10
Processed zwiebel_train_7.mp4 frame 11
Processed zwiebel_train_7.mp4 frame 12
Processed zwiebel_train_7.mp4 frame 13
Processed zwiebel_train_7.mp4 frame 14
Processed zwiebel_train_7.mp4 frame 15
Processed zwiebel_train_7.mp4 frame 16
Processed zwiebel_train_7.mp4 frame 17
Processed zwiebel_train_7.mp4 frame 18
Processed zwiebel_train_7.mp4 frame 19
Processed zwiebel_train_7.mp4 frame 20
Processed zwiebel_train_7.mp4 frame 21
Processed zwiebel_train_7.mp4 frame 22
Processed zwiebel_train_7.mp4 frame 23
Processed zwiebel_train_7.mp4 frame 24
Processed zwiebel_train_7.mp4 frame 25
Processed zwiebel_train_7.mp4 frame 26
Processed zwiebel_train_7.mp4 frame 27
Processed zwiebel_train_7.mp4 frame 28
Processed zwiebel_train_7.mp4 frame 29
Processed zwiebel_train_7.mp4 frame 30
Processed zwiebel_train_7.mp4

...

I0000 00:00:1750067679.338072 60479919 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1750067679.342824 60483427 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750067679.346750 60483431 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1750067679.351619 60479919 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1750067679.351917 60479919 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1750067679.354525 60483443 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750067679.361367 60483448 inference_feedback_manager.cc:114] Feedback man

Processed zwiebel_train_12.mp4 frame 4
Processed zwiebel_train_12.mp4 frame 5
Processed zwiebel_train_12.mp4 frame 6
Processed zwiebel_train_12.mp4 frame 7
Processed zwiebel_train_12.mp4 frame 8
Processed zwiebel_train_12.mp4 frame 9
Processed zwiebel_train_12.mp4 frame 10
Processed zwiebel_train_12.mp4 frame 11
Processed zwiebel_train_12.mp4 frame 12
Processed zwiebel_train_12.mp4 frame 13
Processed zwiebel_train_12.mp4 frame 14
Processed zwiebel_train_12.mp4 frame 15
Processed zwiebel_train_12.mp4 frame 16
Processed zwiebel_train_12.mp4 frame 17
Processed zwiebel_train_12.mp4 frame 18
Processed zwiebel_train_12.mp4 frame 19
Processed zwiebel_train_12.mp4 frame 20
Processed zwiebel_train_12.mp4 frame 21
Processed zwiebel_train_12.mp4 frame 22
Processed zwiebel_train_12.mp4 frame 23
Processed zwiebel_train_12.mp4 frame 24
Processed zwiebel_train_12.mp4 frame 25
Processed zwiebel_train_12.mp4 frame 26
Processed zwiebel_train_12.mp4 frame 27
Processed zwiebel_train_12.mp4 frame 28
Proces

I0000 00:00:1750067680.850450 60479919 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1750067680.855595 60483541 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750067680.859435 60483548 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1750067680.863671 60479919 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1750067680.863962 60479919 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1750067680.866516 60483556 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750067680.873725 60483555 inference_feedback_manager.cc:114] Feedback man

Processed zwiebel_train_13.mp4 frame 3
Processed zwiebel_train_13.mp4 frame 4
Processed zwiebel_train_13.mp4 frame 5
Processed zwiebel_train_13.mp4 frame 6
Processed zwiebel_train_13.mp4 frame 7
Processed zwiebel_train_13.mp4 frame 8
Processed zwiebel_train_13.mp4 frame 9
Processed zwiebel_train_13.mp4 frame 10
Processed zwiebel_train_13.mp4 frame 11
Processed zwiebel_train_13.mp4 frame 12
Processed zwiebel_train_13.mp4 frame 13
Processed zwiebel_train_13.mp4 frame 14
Processed zwiebel_train_13.mp4 frame 15
Processed zwiebel_train_13.mp4 frame 16
Processed zwiebel_train_13.mp4 frame 17
Processed zwiebel_train_13.mp4 frame 18
Processed zwiebel_train_13.mp4 frame 19
Processed zwiebel_train_13.mp4 frame 20
Processed zwiebel_train_13.mp4 frame 21
Processed zwiebel_train_13.mp4 frame 22
Processed zwiebel_train_13.mp4 frame 23
Processed zwiebel_train_13.mp4 frame 24
Processed zwiebel_train_13.mp4 frame 25
Processed zwiebel_train_13.mp4 frame 26
Processed zwiebel_train_13.mp4 frame 27
Process

OpenCV: Couldn't read video stream from file "../food/data/train/zwiebel/landmarks/zwiebel_train_4.mp4_landmarks.csv"
I0000 00:00:1750067682.025975 60479919 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1750067682.031358 60483607 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750067682.035541 60483607 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1750067682.040091 60479919 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Max
W0000 00:00:1750067682.040499 60479919 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1750067682.043336 60483624 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disablin